In [1]:
import subprocess

import pandas as pd
import numpy as np
import dask.dataframe as dd
import matplotlib.pyplot as plt
import seaborn as sns


/Users/wesmadrigal/projects/graphreduce/venv/lib/python3.9/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 8.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


In [8]:
files = [
    'https://raw.githubusercontent.com/wesmadrigal/GraphReduce/master/tests/data/cust_data/cust.csv',
    'https://raw.githubusercontent.com/wesmadrigal/GraphReduce/master/tests/data/cust_data/notification_interaction_types.csv',
    'https://raw.githubusercontent.com/wesmadrigal/GraphReduce/master/tests/data/cust_data/notification_interactions.csv',
    'https://raw.githubusercontent.com/wesmadrigal/GraphReduce/master/tests/data/cust_data/notifications.csv',
    'https://raw.githubusercontent.com/wesmadrigal/GraphReduce/master/tests/data/cust_data/order_products.csv',
    'https://raw.githubusercontent.com/wesmadrigal/GraphReduce/master/tests/data/cust_data/orders.csv'
]

In [23]:
for file in files:
  subprocess.run(['wget', file])

In [24]:
!ls | grep .csv

cust.csv
notification_interactions.csv
notification_interaction_types.csv
notifications.csv
order_products.csv
orders.csv


# Load the data frames and join the files together at the Customer level

In [25]:
cust = pd.read_csv('cust.csv')
orders = pd.read_csv('orders.csv')
notif = pd.read_csv('notifications.csv')
ni = pd.read_csv('notification_interactions.csv')
nit = pd.read_csv('notification_interaction_types.csv')
op = pd.read_csv('order_products.csv')


In [26]:
cust.head(2)

,id,name
0,1,wes
1,2,john


In [27]:
# 5 joins:

# notification interaction types references notification interactions
# notification interactions references notifications
# notifications references customers
# order products references orders
# orders references customers

## Merge notification_interaction_types.csv with notification_interactions.csv

In [28]:
ni.head(2)

,id,notification_id,interaction_type_id,ts
0,1000,101,1500,2022-08-06
1,1001,101,1600,2022-08-07


In [29]:
len(ni)

18

In [30]:

ni = ni.merge(nit, left_on='interaction_type_id',
              right_on='id',
                how='left', suffixes=('', '_y'))

ni.drop(ni.filter(regex='_y$').columns, axis=1, inplace=True)

In [31]:
ni.head(2)

,id,notification_id,interaction_type_id,ts,name
0,1000,101,1500,2022-08-06,viewed
1,1001,101,1600,2022-08-07,opened


In [32]:
len(ni)

18

## Merge notification_interactions.csv with notifications.csv

In [33]:
notif.head(2)

,id,customer_id,ts
0,101,1,2022-08-05
1,102,1,2023-01-01


In [34]:
len(notif)

17

In [35]:
notif = notif.merge(ni,
           left_on='id',
           right_on='notification_id',
           suffixes=('','_y'),
           how='left'
           )


In [36]:
notif.drop(notif.filter(regex='_y$').columns, axis=1, inplace=True)

In [37]:
notif.head(2)

,id,customer_id,ts,notification_id,interaction_type_id,name
0,101,1,2022-08-05,101.0,1500.0,viewed
1,101,1,2022-08-05,101.0,1600.0,opened


In [38]:
len(notif)

25

## Merge notifications.csv with cust.csv

In [39]:
len(cust)

4

In [40]:
cust.head(2)

,id,name
0,1,wes
1,2,john


In [41]:
cust = cust.merge(notif,
           left_on='id',
           right_on='customer_id',
           suffixes=('','_y'),
           how='left'
           )
cust.drop(cust.filter(regex='_y$').columns, axis=1, inplace=True)

In [42]:
len(cust)

25

In [43]:
cust.head(2)

,id,name,customer_id,ts,notification_id,interaction_type_id
0,1,wes,1,2022-08-05,101.0,1500.0
1,1,wes,1,2022-08-05,101.0,1600.0


## Merge order_products.csv with orders.csv

In [44]:
op.head(2)

,id,order_id,product_id
0,1,1,1
1,2,1,2


In [45]:
orders.head(2)

,id,customer_id,ts
0,1,1,2023-05-12
1,2,1,2023-06-01


In [46]:
len(orders)

9

In [47]:
orders = orders.merge(
    op,
    how='left',
    left_on='id',
    right_on='order_id',
    suffixes=('','_y')
)

In [48]:
len(orders)

22

In [49]:
orders.drop(orders.filter(regex='_y$').columns, axis=1, inplace=True)

In [50]:
orders.head(2)

,id,customer_id,ts,order_id,product_id
0,1,1,2023-05-12,1.0,1.0
1,1,1,2023-05-12,1.0,2.0


## Merge orders.csv with cust.csv

In [51]:
cust.merge(orders,
          how='left',
          left_on='id',
          right_on='customer_id',
          suffixes=('','_y'))

,id,name,customer_id,ts,notification_id,interaction_type_id,id_y,customer_id_y,ts_y,order_id,product_id
0,1,wes,1,2022-08-05,101.0,1500.0,1,1,2023-05-12,1.0,1.0
1,1,wes,1,2022-08-05,101.0,1500.0,1,1,2023-05-12,1.0,2.0
2,1,wes,1,2022-08-05,101.0,1500.0,1,1,2023-05-12,1.0,3.0
3,1,wes,1,2022-08-05,101.0,1500.0,1,1,2023-05-12,1.0,4.0
4,1,wes,1,2022-08-05,101.0,1500.0,2,1,2023-06-01,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
204,3,ryan,3,2023-09-01,NaN,NaN,5,3,2023-06-01,5.0,2.0
205,4,tianji,4,2024-02-01,NaN,NaN,8,4,2024-01-01,NaN,NaN
206,4,tianji,4,2024-02-01,NaN,NaN,9,4,2024-02-01,NaN,NaN
207,4,tianji,4,2024-02-15,NaN,NaN,8,4,2024-01-01,NaN,NaN


In [52]:
cust = cust.merge(orders,
          how='left',
          left_on='id',
          right_on='customer_id',
          suffixes=('','_y'))
cust.drop(cust.filter(regex='_y$').columns, axis=1, inplace=True)

In [53]:
cust.head(2)

,id,name,customer_id,ts,notification_id,interaction_type_id,order_id,product_id
0,1,wes,1,2022-08-05,101.0,1500.0,1.0,1.0
1,1,wes,1,2022-08-05,101.0,1500.0,1.0,2.0


In [54]:
len(cust)

209

## This isn't right...
## There shouldn't be 209 rows there are only 4 customers
## We have to be sure to aggregate before each join

In [55]:
# notification_interaction_types.csv and join to notification_interactions.csv

In [58]:
cust = pd.read_csv('cust.csv')
orders = pd.read_csv('orders.csv')
notif = pd.read_csv('notifications.csv')
ni = pd.read_csv('notification_interactions.csv')
nit = pd.read_csv('notification_interaction_types.csv')
op = pd.read_csv('order_products.csv')


In [59]:
len(nit)

3

In [60]:
len(ni)

18

In [61]:
ni = ni.merge(nit, left_on='interaction_type_id',
              right_on='id',
                how='left', suffixes=('', '_y'))

ni.drop(ni.filter(regex='_y$').columns, axis=1, inplace=True)

In [62]:
len(ni)


18

In [63]:
# notification_interaction_types.csv doesn't need to be aggregated

In [64]:
# aggregate notification_interactions.csv and join to notifications.csv

In [65]:
ni.head(2)

,id,notification_id,interaction_type_id,ts,name
0,1000,101,1500,2022-08-06,viewed
1,1001,101,1600,2022-08-07,opened


In [66]:
ni.groupby('notification_id').agg(**{
    'num_interactions': pd.NamedAgg(column='id', aggfunc='count'),
    'first_interaction':pd.NamedAgg(column='ts', aggfunc='min'),
    'last_interaction':pd.NamedAgg(column='ts',aggfunc='max')
}).reset_index().head()

,notification_id,num_interactions,first_interaction,last_interaction
0,101,3,2022-08-06,2022-08-08
1,102,3,2023-01-01,2023-01-03
2,103,3,2023-05-05,2023-05-07
3,104,2,2023-05-10,2023-05-11
4,105,2,2023-05-11,2023-05-11


In [67]:
notif.head(2)

,id,customer_id,ts
0,101,1,2022-08-05
1,102,1,2023-01-01


In [68]:
len(notif)

17

In [69]:
notif = notif.merge(
        ni.groupby('notification_id').agg(**{
        'num_interactions': pd.NamedAgg(column='id', aggfunc='count'),
        'first_interaction':pd.NamedAgg(column='ts', aggfunc='min'),
        'last_interaction':pd.NamedAgg(column='ts',aggfunc='max')
        }).reset_index(),
        how='left',
        left_on='id',
        right_on='notification_id'
)

In [70]:
len(notif)

17

In [71]:
notif.head(2)


,id,customer_id,ts,notification_id,num_interactions,first_interaction,last_interaction
0,101,1,2022-08-05,101.0,3.0,2022-08-06,2022-08-08
1,102,1,2023-01-01,102.0,3.0,2023-01-01,2023-01-03


In [72]:
# aggregate notifications and join to customers

In [73]:
notif.groupby('customer_id').agg(**{
    'num_notifications': pd.NamedAgg(column='notification_id',aggfunc='count'),
    'num_interactions': pd.NamedAgg(column='num_interactions', aggfunc='sum'),
    'first_notification': pd.NamedAgg(column='ts', aggfunc='min'),
    'last_notification':pd.NamedAgg(column='ts',aggfunc='max')
}).reset_index()

,customer_id,num_notifications,num_interactions,first_notification,last_notification
0,1,6,14.0,2022-08-05,2023-06-23
1,2,4,4.0,2022-09-05,2023-05-22
2,3,0,0.0,2023-06-12,2023-09-01
3,4,0,0.0,2024-02-01,2024-02-15


In [74]:
cust = cust.merge(
    notif.groupby('customer_id').agg(**{
    'num_notifications': pd.NamedAgg(column='notification_id',aggfunc='count'),
    'num_interactions': pd.NamedAgg(column='num_interactions', aggfunc='sum'),
    'first_notification': pd.NamedAgg(column='ts', aggfunc='min'),
    'last_notification':pd.NamedAgg(column='ts',aggfunc='max')
}).reset_index(),
    how='left',
    left_on='id',
    right_on='customer_id'
)

In [75]:
len(cust)

4

In [76]:
cust.head(2)

,id,name,customer_id,num_notifications,num_interactions,first_notification,last_notification
0,1,wes,1,6,14.0,2022-08-05,2023-06-23
1,2,john,2,4,4.0,2022-09-05,2023-05-22


In [77]:
# aggregate order_products.csv and join to orders.csv

In [78]:
len(op)

19

In [79]:
len(orders)

9

In [80]:
op.head(2)

,id,order_id,product_id
0,1,1,1
1,2,1,2


In [81]:
op.groupby('order_id').agg(**{
    'num_products': pd.NamedAgg(column='product_id',aggfunc='count')
}).reset_index().head()

,order_id,num_products
0,1,4
1,2,4
2,3,4
3,4,4
4,5,1


In [82]:
orders = orders.merge(
    op.groupby('order_id').agg(**{
    'num_products': pd.NamedAgg(column='product_id',aggfunc='count')
    }).reset_index(),
    how='left',
    left_on='id',
    right_on='order_id'
)

In [83]:
len(orders)

9

In [84]:
# aggregate orders.csv and join to cust.csv

In [85]:
orders.head(2)

,id,customer_id,ts,order_id,num_products
0,1,1,2023-05-12,1.0,4.0
1,2,1,2023-06-01,2.0,4.0


In [86]:
orders.groupby('customer_id').agg(**{
    'num_orders': pd.NamedAgg(column='order_id',aggfunc='count'),
    'num_products_ordered': pd.NamedAgg(column='num_products', aggfunc='sum'),
    'first_order': pd.NamedAgg(column='ts', aggfunc='min'),
    'last_order': pd.NamedAgg(column='ts', aggfunc='max')
}).reset_index()

,customer_id,num_orders,num_products_ordered,first_order,last_order
0,1,3,10.0,2023-05-12,2023-09-02
1,2,2,8.0,2022-08-05,2023-10-15
2,3,1,1.0,2023-06-01,2023-06-01
3,4,0,0.0,2024-01-01,2024-02-01


In [87]:
len(cust)

4

In [88]:
cust = cust.merge(
    orders.groupby('customer_id').agg(**{
    'num_orders': pd.NamedAgg(column='order_id',aggfunc='count'),
    'num_products_ordered': pd.NamedAgg(column='num_products', aggfunc='sum'),
    'first_order': pd.NamedAgg(column='ts', aggfunc='min'),
    'last_order': pd.NamedAgg(column='ts', aggfunc='max')
}).reset_index(),
    how='left',
    left_on='id',
    right_on='customer_id'
)

In [89]:
len(cust)


4

In [90]:
cust.head()

,id,name,customer_id_x,num_notifications,num_interactions,first_notification,last_notification,customer_id_y,num_orders,num_products_ordered,first_order,last_order
0,1,wes,1,6,14.0,2022-08-05,2023-06-23,1,3,10.0,2023-05-12,2023-09-02
1,2,john,2,4,4.0,2022-09-05,2023-05-22,2,2,8.0,2022-08-05,2023-10-15
2,3,ryan,3,0,0.0,2023-06-12,2023-09-01,3,1,1.0,2023-06-01,2023-06-01
3,4,tianji,4,0,0.0,2024-02-01,2024-02-15,4,0,0.0,2024-01-01,2024-02-01


## How about filtering dates?
## What if I want to train models up until Jan 2024 and predict from Jan 2024 - now?

## Issues encountered
* prefixing data, where do all the columns come from?
* data duplication - 1:many, 1:1, many:1 relationships
* time
* repetitive operations such as joins


In [92]:
# OPEN IN A NEW NOTEBOOK: https://github.com/wesmadrigal/GraphReduce/blob/master/examples/ODSC_East_2024_ex2.ipynb

OPEN IN A NEW NOTEBOOK: